In [1]:
!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cu121
!pip install triton==2.1.0
!pip install bitsandbytes==0.41.1
!pip install unsloth

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 73.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch, triton, bitsandbytes as bnb, unsloth
print("TORCH:", torch.__version__)
print("TRITON:", triton.__version__)
print("BNB:", bnb.__version__)
print("UNSLOTH:", unsloth.__version__)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-21 19:28:48.199510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763753328.582378      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763753328.720928      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
TORCH: 2.9.0+cu128
TRITON: 3.5.0
BNB: 0.48.2
UNSLOTH: 2025.11.3


In [11]:
import torch
import torch.nn as nn
import triton
import triton.language as tl


@triton.jit
def _quantize_rowwise_int4(
    w_ptr, packed_ptr, scale_ptr,
    rows, cols,
    stride_wm, stride_wn,
    stride_pm, stride_pn,
    BLOCK_SIZE: tl.constexpr,
):
    row_id = tl.program_id(0)
    if row_id >= rows:
        return

    row_w = w_ptr + row_id * stride_wm

    max_abs = 0.0
    for start in range(0, tl.cdiv(cols, BLOCK_SIZE)):
        offs = start * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
        mask = offs < cols
        vals = tl.load(row_w + offs * stride_wn, mask=mask, other=0.0)
        max_abs = tl.maximum(max_abs, tl.max(tl.abs(vals), axis=0))

    scale = tl.where(max_abs > 0, max_abs / 7.0, 1.0)
    tl.store(scale_ptr + row_id, scale.to(tl.float32))

    packed_cols = tl.cdiv(cols, 2)

    for start in range(0, tl.cdiv(packed_cols, BLOCK_SIZE)):
        offs = start * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
        mask = offs < packed_cols

        even_idx = offs * 2
        odd_idx = even_idx + 1

        even_mask = mask & (even_idx < cols)
        odd_mask  = mask & (odd_idx < cols)

        even_vals = tl.load(row_w + even_idx * stride_wn, mask=even_mask, other=0.0)
        odd_vals  = tl.load(row_w + odd_idx  * stride_wn, mask=odd_mask,  other=0.0)


        xe = even_vals / scale
        xe = tl.minimum(tl.maximum(xe, -8.0), 7.0)
        sign_e = tl.where(xe >= 0, 1.0, -1.0)
        even_q = sign_e * tl.floor(tl.abs(xe) + 0.5)
        
        xo = odd_vals / scale
        xo = tl.minimum(tl.maximum(xo, -8.0), 7.0)
        sign_o = tl.where(xo >= 0, 1.0, -1.0)
        odd_q = sign_o * tl.floor(tl.abs(xo) + 0.5)
        

        even_u = (even_q + 8).to(tl.uint8)
        odd_u  = (odd_q  + 8).to(tl.uint8)

        packed_val = even_u | (odd_u << 4)

        col_p = packed_ptr + offs * stride_pm
        tl.store(col_p + row_id * stride_pn, packed_val, mask=mask)


def quantize_rowwise_int4(weight: torch.Tensor):
    weight = weight.contiguous().to(torch.float16)
    rows, cols = weight.shape
    packed_cols = (cols + 1) // 2

    packed = torch.empty((packed_cols, rows), dtype=torch.uint8, device=weight.device)
    scales = torch.empty(rows, dtype=torch.float32, device=weight.device)

    grid = (rows,)
    _quantize_rowwise_int4[grid](
        weight, packed, scales,
        rows, cols,
        weight.stride(0), weight.stride(1),
        packed.stride(0), packed.stride(1),
        BLOCK_SIZE=128,
    )
    return packed, scales


@triton.jit
def _matmul_fp16_int4(
    a_ptr, b_ptr, scale_ptr, c_ptr,
    M, N, K,
    stride_am, stride_ak,
    stride_bm, stride_bn,
    stride_cm, stride_cn,
    BLOCK_M: tl.constexpr, BLOCK_N: tl.constexpr, BLOCK_K: tl.constexpr,
):
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)

    offs_m = pid_m * BLOCK_M + tl.arange(0, BLOCK_M)
    offs_n = pid_n * BLOCK_N + tl.arange(0, BLOCK_N)

    mask_m = offs_m < M
    mask_n = offs_n < N

    acc = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float32)

    packed_cols = tl.cdiv(K, 2)

    for k0 in range(0, K, BLOCK_K):
        k = k0 + tl.arange(0, BLOCK_K)
        mask_k = k < K

        a_tile = tl.load(
            a_ptr + offs_m[:, None] * stride_am + k[None, :] * stride_ak,
            mask=mask_m[:, None] & mask_k[None, :],
            other=0.0,
        ).to(tl.float32)

        pack_idx = k // 2
        mask_pack = pack_idx < packed_cols

        w_pack = tl.load(
            b_ptr + pack_idx[None, :] * stride_bm + offs_n[:, None] * stride_bn,
            mask=mask_n[:, None] & mask_pack[None, :],
            other=0,
        ).to(tl.uint8)

        low  = (w_pack & 0x0F).to(tl.float32) - 8.0
        high = ((w_pack >> 4) & 0x0F).to(tl.float32) - 8.0
        w_tile = tl.where((k % 2 == 1)[None, :], high, low)

        scales = tl.load(scale_ptr + offs_n, mask=mask_n, other=1.0).to(tl.float32)
        w_tile = w_tile * scales[:, None]

        acc += tl.dot(a_tile, tl.trans(w_tile))

    tl.store(
        c_ptr + offs_m[:, None] * stride_cm + offs_n[None, :] * stride_cn,
        acc.to(tl.float16),
        mask=mask_m[:, None] & mask_n[None, :],
    )


def matmul_fp16_int4(a: torch.Tensor, b_packed: torch.Tensor, scales: torch.Tensor):
    a = a.contiguous().to(torch.float16)
    M, K = a.shape
    packed_K, N = b_packed.shape

    assert packed_K >= (K + 1) // 2

    c = torch.empty((M, N), dtype=torch.float16, device=a.device)

    grid = (triton.cdiv(M, 64), triton.cdiv(N, 64))
    _matmul_fp16_int4[grid](
        a, b_packed, scales,
        c, M, N, K,
        a.stride(0), a.stride(1),
        b_packed.stride(0), b_packed.stride(1),
        c.stride(0), c.stride(1),
        BLOCK_M=64, BLOCK_N=64, BLOCK_K=64,
    )
    return c


class QuantLinear(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = False):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        packed_cols = (in_features + 1) // 2

        self.register_buffer(
            "weight_packed",
            torch.zeros((packed_cols, out_features), dtype=torch.uint8),
        )
        self.register_buffer(
            "weight_scale",
            torch.ones(out_features, dtype=torch.float32),
        )

        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features, dtype=torch.float16))
        else:
            self.register_parameter("bias", None)

    @torch.no_grad()
    def quantize_from_fp16(self, weight: torch.Tensor, bias: torch.Tensor | None = None):
        weight = weight.reshape(self.out_features, self.in_features)
        packed, scales = quantize_rowwise_int4(weight)
        self.weight_packed.copy_(packed)
        self.weight_scale.copy_(scales)

        if self.bias is not None and bias is not None:
            self.bias.copy_(bias.to(torch.float16))

    def forward(self, x: torch.Tensor):
        orig = x.shape[:-1]
        x = x.reshape(-1, self.in_features)
        out = matmul_fp16_int4(x, self.weight_packed, self.weight_scale)

        if self.bias is not None:
            out = out + self.bias

        return out.reshape(*orig, self.out_features)

In [12]:
TARGETS = {"q_proj", "k_proj", "v_proj", "o_proj",
           "gate_proj", "up_proj", "down_proj"}

def replace_linears_with_quant(module):
    for name, child in list(module.named_children()):

        # проверка по имени
        if name in TARGETS and isinstance(child, torch.nn.Linear):

            out_f, in_f = child.weight.shape

            q = QuantLinear(in_f, out_f, child.bias is not None).cuda()
            q.quantize_from_fp16(child.weight, child.bias)

            setattr(module, name, q)
            continue

        replace_linears_with_quant(child)


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "unsloth/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="cuda",
)

replace_linears_with_quant(model)

In [14]:
prompt = "Who is Albert Einstein?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

out = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(out[0]))

<|begin_of_text|>Who is Albert Einstein? He is a very famous scientist and physicist.
Albert Einstein was a German-born physicist and mathematician. He is one of the most famous scientists and physicists in the history of science. Einstein is not a


In [15]:
import torch
import time

def benchmark_matmuls():
    device = "cuda"

    print("Warmup...")
    Xw = torch.randn(64, 4096, dtype=torch.float16, device=device)
    Ww = torch.randn(4096, 4096, dtype=torch.float16, device=device)
    Wp, Sc = quantize_rowwise_int4(Ww)
    _ = matmul_fp16_int4(Xw, Wp, Sc)
    torch.cuda.synchronize()
    print("Warmup done.\n")

    tests = [
        (128, 4096, 4096),
        (512, 4096, 4096),
        (2048, 4096, 4096),

        (128, 4096, 14336),
        (512, 4096, 14336),
        (2048, 4096, 14336),
    ]

    for B, K, N in tests:
        print(f"=== X: {B}×{K}, W: {N}×{K} ===")

        X = torch.randn(B, K, dtype=torch.float16, device=device)
        W = torch.randn(N, K, dtype=torch.float16, device=device)

        torch.cuda.synchronize()
        t0 = time.time()
        Y_fp16 = X @ W.T
        torch.cuda.synchronize()
        t_fp16 = (time.time() - t0) * 1000

        W_packed, scales = quantize_rowwise_int4(W)

        torch.cuda.synchronize()
        t0 = time.time()
        Y_q = matmul_fp16_int4(X, W_packed, scales)
        torch.cuda.synchronize()
        t_int4 = (time.time() - t0) * 1000

        mae = (Y_fp16 - Y_q).abs().mean().item()
        maxe = (Y_fp16 - Y_q).abs().max().item()

        print(f"fp16:  {t_fp16:.3f} ms")
        print(f"int4:  {t_int4:.3f} ms")
        print(f"speed: ×{t_fp16 / t_int4:.3f}")
        print(f"err:   MAE={mae:.4f}, MAX={maxe:.2f}\n")

In [16]:
benchmark_matmuls()

Warmup...
Warmup done.

=== X: 128×4096, W: 4096×4096 ===
fp16:  0.269 ms
int4:  1.151 ms
speed: ×0.234
err:   MAE=7.9922, MAX=51.81

=== X: 512×4096, W: 4096×4096 ===
fp16:  0.602 ms
int4:  3.433 ms
speed: ×0.176
err:   MAE=8.0156, MAX=58.69

=== X: 2048×4096, W: 4096×4096 ===
fp16:  1.531 ms
int4:  13.701 ms
speed: ×0.112
err:   MAE=8.0156, MAX=60.62

=== X: 128×4096, W: 14336×4096 ===
fp16:  0.676 ms
int4:  3.474 ms
speed: ×0.195
err:   MAE=7.9883, MAX=54.56

=== X: 512×4096, W: 14336×4096 ===
fp16:  1.387 ms
int4:  15.405 ms
speed: ×0.090
err:   MAE=8.0156, MAX=56.75

=== X: 2048×4096, W: 14336×4096 ===
fp16:  6.306 ms
int4:  69.195 ms
speed: ×0.091
err:   MAE=8.0000, MAX=63.06



In [17]:
import torch
from datasets import load_dataset
import math

def batchify_texts(texts, tokenizer, batch_size=8, max_tokens=128):
    batch = []
    for t in texts:
        t = t.strip()
        if not t:
            continue

        enc = tokenizer(
            t,
            return_tensors="pt",
            truncation=True,
            max_length=max_tokens,
        )

        batch.append(enc["input_ids"][0])

        if len(batch) == batch_size:
            padded = torch.nn.utils.rnn.pad_sequence(
                batch, batch_first=True, padding_value=tokenizer.pad_token_id
            )
            yield padded
            batch = []

    if batch:
        padded = torch.nn.utils.rnn.pad_sequence(
            batch, batch_first=True, padding_value=tokenizer.pad_token_id
        )
        yield padded

def fast_ppl(model, tokenizer, batch_size=8, max_tokens=128, limit=None):
    model.gradient_checkpointing_disable()
    model.config.use_cache = False

    ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")["text"]
    texts = ds if limit is None else ds[:limit]

    total_loss = 0.0
    total_tokens = 0

    for batch in batchify_texts(texts, tokenizer, batch_size, max_tokens):
        batch = batch.to("cuda")

        # shift
        labels = batch[:, 1:].clone()
        inputs = batch[:, :-1].clone()

        with torch.no_grad():
            logits = model(input_ids=inputs).logits

        # align shapes
        logits = logits.reshape(-1, logits.size(-1))
        labels = labels.reshape(-1)

        mask = labels != tokenizer.pad_token_id

        loss = torch.nn.functional.cross_entropy(
            logits[mask],
            labels[mask],
            reduction="mean"
        )

        total_loss += loss.item() * mask.sum().item()
        total_tokens += mask.sum().item()

    ppl = math.exp(total_loss / total_tokens)
    return ppl


In [18]:
ppl = fast_ppl(
    model, tokenizer,
    batch_size=8,
    max_tokens=128,
    limit=5000,
)

print("FAST-PPL =", ppl)

FAST-PPL = 68.21869110487442


In [ ]:
from pathlib import Path
save_dir = Path('quantized_model_sym')
save_dir.mkdir(exist_ok=True)
model.save_pretrained(save_dir, safe_serialization=True)
tokenizer.save_pretrained(save_dir)
print('Saved to', save_dir)


In [ ]:
import csv, datetime, pathlib
log_path = pathlib.Path('metrics_log.csv')
log_path.parent.mkdir(parents=True, exist_ok=True)
row = {
    'timestamp': datetime.datetime.utcnow().isoformat(),
    'model': model_name,
    'quant': 'sym',
    'targets': ';'.join(sorted(TARGETS)),
    'ppl': float(ppl),
    'batch_size': 8,
    'max_tokens': 128,
    'limit': 5000,
}
exists = log_path.exists()
with log_path.open('a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=row.keys())
    if not exists:
        writer.writeheader()
    writer.writerow(row)
print('Logged metrics to', log_path)
print(row)
